In [31]:
import pandas as pd

In [32]:
def extract_excels():
    try:
        path =  'preprocessing/preprocessed files/'

        df_Airbnb = pd.read_excel(path + 'Final_Airbnb.xlsx')
        print("df_Airbnb succesfully loaded!")   

        df_Booking = pd.read_excel(path + 'Final_Booking.xlsx')
        print("df_Booking succesfully loaded!")  

    except Exception as err:
        print('Error while connecting to the db')
        print(err) 

    return df_Airbnb, df_Booking

In [33]:
df_Airbnb, df_Booking = extract_excels()
merged = pd.concat([df_Airbnb, df_Booking])
merged.head()

df_Airbnb succesfully loaded!
df_Booking succesfully loaded!


,Unnamed: 0,Status,Guest name,Origin,# of guests,Booked,Start date,End date,# of nights,# of days pre booked,Listing name,# of beds,Earnings,Tax,Earnings after Tax,Advertiser
0,0,Confirmed,Egor Zhidkov,Russian Federation,2,2019-03-28,2019-06-20,2019-06-22,2,84,Studio with Patio,2,67.90,4.0,63.90,Airbnb
1,1,Confirmed,Youstina Daoud,Egypt,2,2019-03-30,2019-06-03,2019-06-05,2,65,Studio with Patio,2,67.90,4.0,63.90,Airbnb
2,2,Confirmed,Öznur Balaban,Turkey,2,2019-04-09,2019-06-08,2019-06-09,1,60,Studio with Patio,2,33.95,2.0,31.95,Airbnb
3,3,Confirmed,Михаил Кодолов,Russian Federation,2,2019-05-01,2019-06-16,2019-06-20,4,46,Studio with Patio,2,135.80,8.0,127.80,Airbnb
4,4,Confirmed,Janie Macpherson,United States,1,2019-05-02,2019-09-18,2019-09-23,5,139,Studio with Patio,2,169.75,5.0,164.75,Airbnb


In [34]:
merged = merged.sort_values('Start date')
merged.head()

,Unnamed: 0,Status,Guest name,Origin,# of guests,Booked,Start date,End date,# of nights,# of days pre booked,Listing name,# of beds,Earnings,Tax,Earnings after Tax,Advertiser
1,1,Confirmed,Youstina Daoud,Egypt,2,2019-03-30,2019-06-03,2019-06-05,2,65,Studio with Patio,2,67.90,4.0,63.90,Airbnb
6,6,Confirmed,Kristina Holm Jensen,Denmark,2,2019-05-05,2019-06-05,2019-06-07,2,31,Studio with Patio,2,67.90,4.0,63.90,Airbnb
2,2,Confirmed,Öznur Balaban,Turkey,2,2019-04-09,2019-06-08,2019-06-09,1,60,Studio with Patio,2,33.95,2.0,31.95,Airbnb
10,10,Confirmed,Steve Qj,United States,1,2019-06-01,2019-06-10,2019-06-11,1,9,Studio with Patio,2,33.95,1.0,32.95,Airbnb
5,5,Confirmed,Quinten Spakman,Netherlands,2,2019-05-04,2019-06-13,2019-06-15,2,40,Studio with Patio,2,67.90,4.0,63.90,Airbnb


In [35]:
df = pd.DataFrame()
Dim_Apartment = pd.concat([df, pd.DataFrame(columns=['apartment_id', 'apartment_name'])], axis=1)
Dim_Advertiser = pd.concat([df, pd.DataFrame(columns=['advertiser_id', 'advertiser_name'])], axis=1)
Dim_Time = pd.concat([df, pd.DataFrame(columns=['time_id', 'date', 'day', 'month', 'year'])], axis=1)
Fct_Cashflow = pd.concat([df, pd.DataFrame(columns=['cashflow_id', 'apartment_id', 'advertiser_id', 'time_id', 'earnings_for_day'])], axis=1)

In [36]:
merged["apartment_id"] = merged["Listing name"].rank(method='dense').astype(int)

Dim_Apartment = merged[['apartment_id', 'Listing name']].drop_duplicates()

Dim_Apartment

,apartment_id,Listing name
1,3,Studio with Patio
22,2,One-Bedroom Apartment with Patio and Sea View
23,4,Studio with Patio and Sea View
7,1,One-Bedroom Apartment with Balcony and Sea View


In [37]:
merged["advertiser_id"] = merged["Advertiser"].rank(method='dense').astype(int)

Dim_Advertiser = merged[['advertiser_id', 'Advertiser']].drop_duplicates()

Dim_Advertiser

,advertiser_id,Advertiser
1,1,Airbnb
22,2,Booking


In [38]:
merged = merged.loc[merged["Status"] == "Confirmed"]
merged["Price per day"] = merged["Earnings after Tax"] / merged["# of nights"]
merged = merged[["Start date", "End date", "apartment_id" , "advertiser_id", "Earnings after Tax", "# of nights", "Price per day"]]
merged

,Start date,End date,apartment_id,advertiser_id,Earnings after Tax,# of nights,Price per day
1,2019-06-03,2019-06-05,3,1,63.90,2,31.95
6,2019-06-05,2019-06-07,3,1,63.90,2,31.95
2,2019-06-08,2019-06-09,3,1,31.95,1,31.95
10,2019-06-10,2019-06-11,3,1,32.95,1,32.95
5,2019-06-13,2019-06-15,3,1,63.90,2,31.95
...,...,...,...,...,...,...,...
106,2022-10-03,2022-10-05,1,1,81.36,2,40.68
97,2022-10-03,2022-10-09,2,1,244.08,6,40.68
101,2022-10-06,2022-10-10,1,1,162.72,4,40.68
105,2022-10-07,2022-10-10,4,1,95.85,3,31.95


In [39]:
merged['date'] = merged.apply(lambda x: pd.date_range(start=x['Start date'], end=x['End date'] - pd.Timedelta(days=1)), axis=1)
merged.head()

,Start date,End date,apartment_id,advertiser_id,Earnings after Tax,# of nights,Price per day,date
1,2019-06-03,2019-06-05,3,1,63.90,2,31.95,"DatetimeIndex(['2019-06-03', '2019-06-04'], dt..."
6,2019-06-05,2019-06-07,3,1,63.90,2,31.95,"DatetimeIndex(['2019-06-05', '2019-06-06'], dt..."
2,2019-06-08,2019-06-09,3,1,31.95,1,31.95,"DatetimeIndex(['2019-06-08'], dtype='datetime6..."
10,2019-06-10,2019-06-11,3,1,32.95,1,32.95,"DatetimeIndex(['2019-06-10'], dtype='datetime6..."
5,2019-06-13,2019-06-15,3,1,63.90,2,31.95,"DatetimeIndex(['2019-06-13', '2019-06-14'], dt..."


In [40]:
merged = merged.explode('date')
merged = merged.drop(['Start date', 'End date', 'Earnings after Tax', '# of nights'], axis=1)

merged


,apartment_id,advertiser_id,Price per day,date
1,3,1,31.95,2019-06-03
1,3,1,31.95,2019-06-04
6,3,1,31.95,2019-06-05
6,3,1,31.95,2019-06-06
2,3,1,31.95,2019-06-08
...,...,...,...,...
105,4,1,31.95,2022-10-07
105,4,1,31.95,2022-10-08
105,4,1,31.95,2022-10-09
99,4,1,31.95,2022-10-10
